# `snopy` - basic usage and warehouse management

## Imports

In [1]:
import os
from snopy import SnowflakeConnector

## Credentials

In [2]:
sf_username=os.environ['SNOWFLAKE_USER']
sf_password=os.environ['SNOWFLAKE_PASSWORD']
sf_account=os.environ['SNOWFLAKE_ACCOUNT']

## Connecting to your Snowflake account

In [3]:
sc = SnowflakeConnector(
    username=sf_username,
    password=sf_password,
    account=sf_account
)

## Environment setup

In [4]:
sc.set_environment(
    database='SNOWFLAKE_SAMPLE_DATA',
    schema='TPCH_SF100'
)

In [5]:
# Warehouse is missing!
sc.get_environment()

{'role': 'ACCOUNTADMIN',
 'database': 'SNOWFLAKE_SAMPLE_DATA',
 'schema': 'TPCH_SF100',
 'warehouse': None}

## Creating a virtual warehouse for compute purposes

In [6]:
sc.warehouse.create(
    'TEST_WH',
    or_replace=True,
    warehouse_size='LARGE',
    min_cluster_count=1,
    max_cluster_count=2,
    scaling_policy='ECONOMY'
)

{'results': [('Warehouse TEST_WH successfully created.',)],
 'description': [ResultMetadata(name='status', type_code=2, display_size=None, internal_size=16777216, precision=None, scale=None, is_nullable=True)],
 'statement': 'CREATE OR REPLACE WAREHOUSE TEST_WH WITH WAREHOUSE_SIZE = LARGE MAX_CLUSTER_COUNT = 2 MIN_CLUSTER_COUNT = 1 SCALING_POLICY = ECONOMY AUTO_SUSPEND = 600 AUTO_RESUME = TRUE INITIALLY_SUSPENDED = FALSE'}

## Using virtual warehouse for querying and DML statements

In fact, DDL statements are free of charge in Snowflake ecosystem (which includes creating, altering, droping objects like warehouses, tables, etc.) - [see the documentation for more](https://docs.snowflake.com/en/user-guide/warehouses-overview.html#impact-on-credit-usage-and-billing)

In [7]:
sc.warehouse.use('TEST_WH')
sc.warehouse.get_current()

'TEST_WH'

In [8]:
sc.query_pd('SELECT * FROM CUSTOMER WHERE C_NATIONKEY=23').head(5)

,C_CUSTKEY,C_NAME,C_ADDRESS,C_NATIONKEY,C_PHONE,C_ACCTBAL,C_MKTSEGMENT,C_COMMENT
0,6750022,Customer#006750022,ZDZAqKxQ GLjI mZ9qR1,23,33-465-870-5251,9326.27,AUTOMOBILE,"ar, ironic dolphins. slyly final courts engage..."
1,6750046,Customer#006750046,PnwRZmmdKrJtzqaOEdn3Hjp25,23,33-876-662-4662,788.25,FURNITURE,fluffily unusual asymptotes. quickly regular pla
2,6750093,Customer#006750093,JtlYbCHMfNmxV3UAxdY1mALF2QSuX,23,33-139-998-8416,1928.52,BUILDING,ructions. furiously regular asymptotes wake sl...
3,6750114,Customer#006750114,frEJqHOI0g42tYG2X5jnrqkXmWgz,23,33-322-891-6512,9185.07,HOUSEHOLD,rts. carefully regular requests x-ray blithely...
4,6750121,Customer#006750121,enw4SW7Ke0Qg5Ut153l,23,33-509-421-5704,2442.01,BUILDING,ngage. express requests sleep blithely about t...


## Closing the connection

In [9]:
sc.close_connection()